In [145]:
import numpy as np
import pandas as pd
from sklearn import svm

# 数据读取

In [146]:
train_data = pd.read_csv( 'train.csv' )
test_data = pd.read_csv( 'test_noLabel.csv' )
test_data_origin = test_data.copy()

In [147]:
print( train_data.shape, test_data.shape )

(10000, 15) (3000, 14)


In [148]:
train_data.head()

,ID,sale_date,price,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude
0,0,20150302,54.50,3,2.25,1670,6240,1.0,8,1240,430,1974,0,47.6413,-122.113
1,1,20150211,78.50,4,2.50,3300,10514,2.0,10,3300,0,1984,0,47.6323,-122.036
2,2,20150107,76.50,3,3.25,3190,5283,2.0,9,3190,0,2007,0,47.5534,-122.002
3,3,20141103,72.00,5,2.50,2900,9525,2.0,9,2900,0,1989,0,47.5442,-122.138
4,4,20140603,44.95,5,2.75,2040,7488,1.0,7,1200,840,1969,0,47.7289,-122.172


In [149]:
test_data.head()

,ID,sale_date,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude
0,10000,20140502,2,1.00,880,6380,1.0,7,880,0,1938,1994,47.6924,-122.322
1,10001,20150213,4,2.50,2880,8833,2.0,7,2880,0,2006,0,47.5388,-121.890
2,10002,20141215,4,2.50,1980,5020,2.0,7,1980,0,2007,0,47.5147,-122.190
3,10003,20140617,3,2.50,2830,6536,2.0,9,2830,0,1989,0,47.7462,-122.172
4,10004,20150410,3,1.75,1900,13600,1.0,8,1900,0,1956,0,47.6163,-122.219


# 数据探索

In [150]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
ID               10000 non-null int64
sale_date        10000 non-null int64
price            10000 non-null float64
num_bedroom      10000 non-null int64
num_bathroom     10000 non-null float64
area_house       10000 non-null int64
area_parking     10000 non-null int64
floor            10000 non-null float64
rating           10000 non-null int64
floorage         10000 non-null int64
area_basement    10000 non-null int64
year_built       10000 non-null int64
year_repair      10000 non-null int64
latitude         10000 non-null float64
longitude        10000 non-null float64
dtypes: float64(5), int64(10)
memory usage: 1.1 MB


In [151]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
ID               3000 non-null int64
sale_date        3000 non-null int64
num_bedroom      3000 non-null int64
num_bathroom     3000 non-null float64
area_house       3000 non-null int64
area_parking     3000 non-null int64
floor            3000 non-null float64
rating           3000 non-null int64
floorage         3000 non-null int64
area_basement    3000 non-null int64
year_built       3000 non-null int64
year_repair      3000 non-null int64
latitude         3000 non-null float64
longitude        3000 non-null float64
dtypes: float64(4), int64(10)
memory usage: 328.2 KB


In [152]:
train_data.describe()

,ID,sale_date,price,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.0000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,2.014392e+07,54.287493,3.367600,2.1168,2082.488400,1.535273e+04,1.502850,7.664800,1791.474900,291.013500,1971.204700,85.209700,47.560629,-122.215845
std,2886.89568,4.440706e+03,37.292577,0.893169,0.7741,922.878916,4.577623e+04,0.543642,1.173873,829.449437,446.641339,29.322957,403.491421,0.138570,0.139739
min,0.00000,2.014050e+07,7.500000,0.000000,0.0000,390.000000,5.720000e+02,1.000000,3.000000,390.000000,0.000000,1900.000000,0.000000,47.159300,-122.519000
25%,2499.75000,2.014072e+07,32.250000,3.000000,1.7500,1430.000000,5.023250e+03,1.000000,7.000000,1190.000000,0.000000,1951.750000,0.000000,47.471375,-122.329000
50%,4999.50000,2.014102e+07,45.070000,3.000000,2.2500,1910.000000,7.590000e+03,1.500000,7.000000,1560.000000,0.000000,1975.000000,0.000000,47.572200,-122.234000
75%,7499.25000,2.015022e+07,64.500000,4.000000,2.5000,2550.000000,1.071700e+04,2.000000,8.000000,2211.500000,560.000000,1997.000000,0.000000,47.678400,-122.127000
max,9999.00000,2.015053e+07,688.500000,10.000000,7.7500,9890.000000,1.651359e+06,3.500000,13.000000,8860.000000,4820.000000,2015.000000,2015.000000,47.777600,-121.315000


In [153]:
test_data.describe()

,ID,sale_date,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude
count,3000.000000,3.000000e+03,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,11499.500000,2.014389e+07,3.384333,2.094833,2060.503000,14370.280667,1.461667,7.616667,1767.408333,293.094667,1970.100000,68.478000,47.557591,-122.211974
std,866.169729,4.429481e+03,0.931438,0.787933,916.445061,34791.442850,0.525474,1.195909,831.594341,436.958537,29.216409,363.242207,0.138699,0.141309
min,10000.000000,2.014050e+07,0.000000,0.000000,380.000000,649.000000,1.000000,4.000000,380.000000,0.000000,1900.000000,0.000000,47.164700,-122.512000
25%,10749.750000,2.014072e+07,3.000000,1.500000,1420.000000,5088.750000,1.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,47.465475,-122.324250
50%,11499.500000,2.014102e+07,3.000000,2.250000,1900.000000,7620.000000,1.000000,7.000000,1540.000000,0.000000,1974.000000,0.000000,47.569450,-122.227000
75%,12249.250000,2.015022e+07,4.000000,2.500000,2510.000000,10521.750000,2.000000,8.000000,2160.000000,572.500000,1995.000000,0.000000,47.676425,-122.124000
max,12999.000000,2.015051e+07,10.000000,7.500000,8000.000000,641203.000000,3.500000,13.000000,7850.000000,2610.000000,2015.000000,2014.000000,47.776800,-121.325000


# 特征工程

In [154]:
#ID列无用，删除
train_data.drop( 'ID', axis=1, inplace=True )
test_data.drop( 'ID', axis=1, inplace=True )

In [155]:
train_data.head()

,sale_date,price,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude
0,20150302,54.50,3,2.25,1670,6240,1.0,8,1240,430,1974,0,47.6413,-122.113
1,20150211,78.50,4,2.50,3300,10514,2.0,10,3300,0,1984,0,47.6323,-122.036
2,20150107,76.50,3,3.25,3190,5283,2.0,9,3190,0,2007,0,47.5534,-122.002
3,20141103,72.00,5,2.50,2900,9525,2.0,9,2900,0,1989,0,47.5442,-122.138
4,20140603,44.95,5,2.75,2040,7488,1.0,7,1200,840,1969,0,47.7289,-122.172


In [156]:
test_data.head()

,sale_date,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude
0,20140502,2,1.00,880,6380,1.0,7,880,0,1938,1994,47.6924,-122.322
1,20150213,4,2.50,2880,8833,2.0,7,2880,0,2006,0,47.5388,-121.890
2,20141215,4,2.50,1980,5020,2.0,7,1980,0,2007,0,47.5147,-122.190
3,20140617,3,2.50,2830,6536,2.0,9,2830,0,1989,0,47.7462,-122.172
4,20150410,3,1.75,1900,13600,1.0,8,1900,0,1956,0,47.6163,-122.219


In [157]:
#将时间列转换为时间格式
train_data['sale_date'] = pd.to_datetime( train_data['sale_date'], format='%Y%m%d' )
test_data['sale_date'] = pd.to_datetime( test_data['sale_date'], format='%Y%m%d' )
print( train_data['sale_date'].dtype, test_data['sale_date'].dtype )

datetime64[ns] datetime64[ns]


In [158]:
train_data.head()

,sale_date,price,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude
0,2015-03-02,54.50,3,2.25,1670,6240,1.0,8,1240,430,1974,0,47.6413,-122.113
1,2015-02-11,78.50,4,2.50,3300,10514,2.0,10,3300,0,1984,0,47.6323,-122.036
2,2015-01-07,76.50,3,3.25,3190,5283,2.0,9,3190,0,2007,0,47.5534,-122.002
3,2014-11-03,72.00,5,2.50,2900,9525,2.0,9,2900,0,1989,0,47.5442,-122.138
4,2014-06-03,44.95,5,2.75,2040,7488,1.0,7,1200,840,1969,0,47.7289,-122.172


In [159]:
test_data.head()

,sale_date,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude
0,2014-05-02,2,1.00,880,6380,1.0,7,880,0,1938,1994,47.6924,-122.322
1,2015-02-13,4,2.50,2880,8833,2.0,7,2880,0,2006,0,47.5388,-121.890
2,2014-12-15,4,2.50,1980,5020,2.0,7,1980,0,2007,0,47.5147,-122.190
3,2014-06-17,3,2.50,2830,6536,2.0,9,2830,0,1989,0,47.7462,-122.172
4,2015-04-10,3,1.75,1900,13600,1.0,8,1900,0,1956,0,47.6163,-122.219


In [160]:
#从时间列提取月、日信息作为新特征
train_data['sale_month'] = train_data['sale_date'].apply( lambda x: x.month )
train_data['sale_day'] = train_data['sale_date'].apply( lambda x: x.day )

test_data['sale_month'] = test_data['sale_date'].apply( lambda x: x.month )
test_data['sale_day'] = test_data['sale_date'].apply( lambda x: x.day )

In [161]:
train_data.head()

,sale_date,price,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude,sale_month,sale_day
0,2015-03-02,54.50,3,2.25,1670,6240,1.0,8,1240,430,1974,0,47.6413,-122.113,3,2
1,2015-02-11,78.50,4,2.50,3300,10514,2.0,10,3300,0,1984,0,47.6323,-122.036,2,11
2,2015-01-07,76.50,3,3.25,3190,5283,2.0,9,3190,0,2007,0,47.5534,-122.002,1,7
3,2014-11-03,72.00,5,2.50,2900,9525,2.0,9,2900,0,1989,0,47.5442,-122.138,11,3
4,2014-06-03,44.95,5,2.75,2040,7488,1.0,7,1200,840,1969,0,47.7289,-122.172,6,3


In [162]:
test_data.head()

,sale_date,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude,sale_month,sale_day
0,2014-05-02,2,1.00,880,6380,1.0,7,880,0,1938,1994,47.6924,-122.322,5,2
1,2015-02-13,4,2.50,2880,8833,2.0,7,2880,0,2006,0,47.5388,-121.890,2,13
2,2014-12-15,4,2.50,1980,5020,2.0,7,1980,0,2007,0,47.5147,-122.190,12,15
3,2014-06-17,3,2.50,2830,6536,2.0,9,2830,0,1989,0,47.7462,-122.172,6,17
4,2015-04-10,3,1.75,1900,13600,1.0,8,1900,0,1956,0,47.6163,-122.219,4,10


In [163]:
#删除时间列
train_data.drop( 'sale_date' ,axis=1, inplace=True )
test_data.drop( 'sale_date' ,axis=1, inplace=True )

In [164]:
train_data.head()

,price,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude,sale_month,sale_day
0,54.50,3,2.25,1670,6240,1.0,8,1240,430,1974,0,47.6413,-122.113,3,2
1,78.50,4,2.50,3300,10514,2.0,10,3300,0,1984,0,47.6323,-122.036,2,11
2,76.50,3,3.25,3190,5283,2.0,9,3190,0,2007,0,47.5534,-122.002,1,7
3,72.00,5,2.50,2900,9525,2.0,9,2900,0,1989,0,47.5442,-122.138,11,3
4,44.95,5,2.75,2040,7488,1.0,7,1200,840,1969,0,47.7289,-122.172,6,3


In [165]:
test_data.head()

,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude,sale_month,sale_day
0,2,1.00,880,6380,1.0,7,880,0,1938,1994,47.6924,-122.322,5,2
1,4,2.50,2880,8833,2.0,7,2880,0,2006,0,47.5388,-121.890,2,13
2,4,2.50,1980,5020,2.0,7,1980,0,2007,0,47.5147,-122.190,12,15
3,3,2.50,2830,6536,2.0,9,2830,0,1989,0,47.7462,-122.172,6,17
4,3,1.75,1900,13600,1.0,8,1900,0,1956,0,47.6163,-122.219,4,10


In [166]:
#数据归一化
price_mean = np.mean( train_data['price'] )
price_std = np.std( train_data['price'] )
train_data = train_data.apply( lambda x: ( x - np.mean(x) ) / np.std( x ) )
test_data = test_data.apply( lambda x: ( x - np.mean(x) ) / np.std( x ) )

In [167]:
train_data.head()

,price,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude,sale_month,sale_day
0,0.005699,-0.411589,0.172079,-0.446981,-0.199081,-0.925012,0.285565,-0.664902,0.311197,0.095333,-0.211192,0.582197,0.736020,-1.143917,-1.575305
1,0.649291,0.708076,0.495051,1.319320,-0.105709,0.914527,1.989411,1.818797,-0.651592,0.436380,-0.211192,0.517245,1.287076,-1.463643,-0.536208
2,0.595658,-0.411589,1.463968,1.200122,-0.219988,0.914527,1.137488,1.686173,-0.651592,1.220787,-0.211192,-0.052170,1.530400,-1.783370,-0.998029
3,0.474984,1.827742,0.495051,0.885872,-0.127316,0.914527,1.137488,1.336526,-0.651592,0.606903,-0.211192,-0.118566,0.557105,1.413894,-1.459849
4,-0.250397,1.827742,0.818024,-0.046041,-0.171817,-0.925012,-0.566359,-0.713129,1.229205,-0.075191,-0.211192,1.214399,0.313782,-0.184738,-1.459849


In [168]:
test_data.head()

,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude,sale_month,sale_day
0,-1.486480,-1.389732,-1.288348,-0.229701,-0.878719,-0.515733,-1.067295,-0.670873,-1.098881,5.301815,0.972119,-0.778749,-0.518438,-1.630092
1,0.661095,0.514300,0.894362,-0.159183,1.024643,-0.515733,1.338125,-0.670873,1.228966,-0.188550,-0.135500,2.278889,-1.484871,-0.343977
2,0.661095,0.514300,-0.087857,-0.268797,1.024643,-0.515733,0.255686,-0.670873,1.263199,-0.188550,-0.309287,0.155529,1.736573,-0.110138
3,-0.412692,0.514300,0.839794,-0.225216,1.024643,1.156914,1.277989,-0.670873,0.647005,-0.188550,1.360075,0.282931,-0.196293,0.123701
4,-0.412692,-0.437716,-0.175166,-0.022144,-0.878719,0.320591,0.159469,-0.670873,-0.482686,-0.188550,0.423357,-0.049729,-0.840582,-0.694736


In [169]:
#分离训练集特征和标签
x_train = train_data.iloc[:, 1:]
y_train = train_data[ 'price' ]

In [170]:
x_train.head()

,num_bedroom,num_bathroom,area_house,area_parking,floor,rating,floorage,area_basement,year_built,year_repair,latitude,longitude,sale_month,sale_day
0,-0.411589,0.172079,-0.446981,-0.199081,-0.925012,0.285565,-0.664902,0.311197,0.095333,-0.211192,0.582197,0.736020,-1.143917,-1.575305
1,0.708076,0.495051,1.319320,-0.105709,0.914527,1.989411,1.818797,-0.651592,0.436380,-0.211192,0.517245,1.287076,-1.463643,-0.536208
2,-0.411589,1.463968,1.200122,-0.219988,0.914527,1.137488,1.686173,-0.651592,1.220787,-0.211192,-0.052170,1.530400,-1.783370,-0.998029
3,1.827742,0.495051,0.885872,-0.127316,0.914527,1.137488,1.336526,-0.651592,0.606903,-0.211192,-0.118566,0.557105,1.413894,-1.459849
4,1.827742,0.818024,-0.046041,-0.171817,-0.925012,-0.566359,-0.713129,1.229205,-0.075191,-0.211192,1.214399,0.313782,-0.184738,-1.459849


In [171]:
y_train.head()

0    0.005699
1    0.649291
2    0.595658
3    0.474984
4   -0.250397
Name: price, dtype: float64

In [172]:
#训练模型
svm_SVR = svm.SVR()
svm_SVR.fit( x_train, y_train )

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [173]:
#预测房价
y_predict = svm_SVR.predict( test_data )
y_predict

array([-0.44047295, -0.01079847, -0.41546805, ..., -0.27116971,
        0.96832184,  0.09203313])

In [174]:
#将归一化的预测房价恢复为正常值
print( 'price_mead: ' + str( price_mean ), 'price_std: ' + str( price_std ) )
y_predict = y_predict * price_std + price_mean
y_predict

price_mead: 54.28749287999999 price_std: 37.29071191235596


array([37.86194308, 53.88481016, 38.79439368, ..., 44.17538132,
       90.39690359, 57.71947392])

In [175]:
print( y_predict.shape )

(3000,)


In [176]:
#按要求保存数据，然后提交到比赛网站
data_save = pd.DataFrame( { 'ID':test_data_origin['ID'], 'price': y_predict } )
data_save.to_csv( './DC_House_Price.csv', index=False )
data_save.head()

,ID,price
0,10000,37.861943
1,10001,53.884810
2,10002,38.794394
3,10003,70.066562
4,10004,68.812917
